# Лабораторная работа по предмету 'основы баз данных'.

* В качестве материала для выполнения данного задания мной был выбран датасет *Литературные произведения в государственных стандартах и программах для средней школы 1998–2022 гг.*, загруженный с сайта dataverse.pushdom.ru
* В наборе данных представлена роспись литературных произведений, вошедших в постсоветские государственные стандарты и программы средней школы по родной литературе (базовый уровень). Хронологические рамки: с 1998 по 2022 г., верхняя граница обусловлена годом выхода первого официального постсоветского документа, содержавшего список предназначенных для изучения в школе произведений.
* Датасет состоит из двух таблиц формата tsv: **curricula.tsv** (с атрибутами source_id, source_year, group, century, author, title, genre, type, oblig_author, oblig_title, grade, section, number_authors, number_titles, profound, source_issue, title_cycle, notes) и **bibliography.tsv** (с атрибутами source_id,  source_year, notes). 


### Шаг 1. Загрузка и импорт необходимых библиотек

In [18]:
import pandas as pd
import subprocess
import mysql.connector
from getpass import getpass
import csv
import re

### Шаг 2. Предварительный анализ содержимого таблиц 

In [2]:
df1 = pd.read_csv('bibliography.tsv', sep='\t')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   source_id    42 non-null     object
 1   source_year  42 non-null     int64 
 2   description  42 non-null     object
 3   notes        1 non-null      object
dtypes: int64(1), object(3)
memory usage: 1.4+ KB


In [3]:
df1.head()

,source_id,source_year,description,notes
0,1998,1998,Временные требования к обязательному минимуму ...,NaN
1,1999,1999,Обязательный минимум содержания среднего (полн...,NaN
2,2004,2004,Федеральный компонент государственного стандар...,"С учетом изменений, внесенных Приказом Минобрн..."
3,2015,2015,Примерная основная образовательная программа о...,NaN
4,2016,2016,Примерная основная образовательная программа с...,NaN


In [4]:
df2 = pd.read_csv('curricula.tsv', sep='\t')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10733 entries, 0 to 10732
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   source_id       10733 non-null  object 
 1   source_year     10733 non-null  int64  
 2   group           3767 non-null   float64
 3   century         10640 non-null  object 
 4   author          10581 non-null  object 
 5   title           10733 non-null  object 
 6   genre           10733 non-null  object 
 7   type            10711 non-null  object 
 8   oblig_author    10581 non-null  object 
 9   oblig_title     8117 non-null   object 
 10  grade           10733 non-null  object 
 11  section         3206 non-null   object 
 12  number_authors  1769 non-null   float64
 13  number_titles   2377 non-null   object 
 14  profound        228 non-null    object 
 15  source_issue    9588 non-null   object 
 16  title_cycle     380 non-null    object 
 17  notes           5264 non-null  

In [5]:
df2.head()

,source_id,source_year,group,century,author,title,genre,type,oblig_author,oblig_title,grade,section,number_authors,number_titles,profound,source_issue,title_cycle,notes
0,2021,2021,9.0,NaN,NaN,Аника-воин,поэзия,отечественная,NaN,нет,6,NaN,NaN,1,NaN,Фольклор,NaN,Не менее 1 баллады народов России и мира по вы...
1,2022–1,2022,10.0,NaN,NaN,Аника-воин,поэзия,отечественная,NaN,нет,6,NaN,NaN,1,NaN,Фольклор,NaN,Не менее 1 баллады народов России и мира по вы...
2,2022–2,2022,392.0,NaN,NaN,Аника-воин,поэзия,отечественная,NaN,нет,6,NaN,NaN,1,NaN,Фольклор,NaN,Не менее 1 баллады народов России и мира по вы...
3,1998,1998,NaN,NaN,NaN,Вольга и Микула Селянинович,поэзия,отечественная,NaN,да,5–9,NaN,NaN,NaN,NaN,Из русского фольклора,NaN,NaN
4,2004,2004,5.0,NaN,NaN,Гэсэр,поэзия,региональная,NaN,нет,5–9,NaN,1.0,1,NaN,Литература народов России,NaN,1 произведение героического эпоса народов Росс...


### Шаг 2.5. Функция для обработки интервальных значений
Как мы видим, в столбцах century, grade и number_titles слишком часто встречаются интервальные значения, такие как '5-9' или '12-13'. Их слишком много, чтобы устранять вручную, поэтому здесь я запишу функцию для последующей обработки подобных ячеек.

In [6]:
# Собственно функция
def process_interval(value):
    # Используем регулярное выражение для извлечения чисел из строки
    numbers = re.findall(r'\d+', value)
    if numbers:
        # Преобразуем числа из строки в целые числа
        numbers = list(map(int, numbers))
        # Возвращаем минимальное и максимальное значения
        return min(numbers), max(numbers)
    else:
        # Если не удалось извлечь числа, возвращаем None
        return None, None

# Проверим функцию на работоспособность
print(process_interval('5–9'))  
print(process_interval('10'))   
print(process_interval(''))   


(5, 9)
(10, 10)
(None, None)


### Шаг 3. Создание (пустых) таблиц

In [7]:
# Подключаемся к MySQL
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password = getpass('Введите ваш пароль!')
)

# Создаем базу данных
cursor = conn.cursor()
cursor.execute("CREATE DATABASE IF NOT EXISTS literature_curricula;")
conn.commit()

# Подключаемся к созданной базе данных
conn = mysql.connector.connect(
     host='localhost',
    user='root',
    password = getpass('Введите ваш пароль!'),
    database="literature_curricula"
)

# Создаем таблицу curricula
cursor = conn.cursor()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS curricula (
        id INT PRIMARY KEY,
        source_id VARCHAR(255),
        source_year INT,
        group_name VARCHAR(255),
        century VARCHAR(255),
        author VARCHAR(255),
        title VARCHAR(255),
        genre VARCHAR(255),
        type VARCHAR(255),
        oblig_author BOOLEAN,
        oblig_title BOOLEAN,
        grade INT,
        section VARCHAR(255),
        number_authors INT,
        number_titles INT,
        profound VARCHAR(255),
        source_issue VARCHAR(255),
        title_cycle VARCHAR(255),
        notes TEXT
    );
""")
conn.commit()

# Создаем таблицу bibliography
cursor.execute("""
    CREATE TABLE IF NOT EXISTS bibliography (
        source_id VARCHAR(255) PRIMARY KEY,
        source_year INT,
        description VARCHAR(10000),
        notes TEXT
    );
""")
conn.commit()

cursor.close()
conn.close()


### Шаг 4. Заполнение таблиц 

In [9]:
# Теперь создадим функцию для загрузки данных из TSV файла в нашу таблицу MySQL

def load_curricula_from_tsv(file_path, table_name):
    with open(file_path, 'r', encoding='utf-8') as tsvfile:
        reader = csv.reader(tsvfile, delimiter='\t')
        next(reader)  # пропускаем заголовки
        year_counts = {}  # Создаем словарь year_counts перед циклом
        for i, row in enumerate(reader):
            if "curricula.tsv" in file_path:
                # Обрабатываем инт. значения в столбцах 'century', 'grade' и 'number_titles' с помощью ранее созданной функции
                row[3], _ = process_interval(row[3])  # century
                row[10], _ = process_interval(row[10])  # grade
                row[13], _ = process_interval(row[13])  # number_titles
            row.insert(0, i)
            # Заменяем 'NA' в столбцах oblig_author и oblig_title на NULL
            row = [None if x == 'NA' else x for x in row]
            # Заменяем 'да' и 'нет' в столбцах oblig_author и oblig_title на True и False
            row[9] = True if row[9] == 'да' else False
            row[10] = True if row[10] == 'да' else False
            placeholders = ', '.join(['%s'] * len(row))
            query = f"INSERT INTO {table_name} VALUES ({placeholders})"
            try:
                cursor.execute(query, tuple(row))
            except:
                print(f"query={query}")
                print(len(placeholders))
                print(f"row={row}")
                print(print(len(row)))
                raise ValueError
        conn.commit()

# Подключаемся к базе данных
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password = getpass('Введите ваш пароль!'),
    database="literature_curricula"
)

cursor = conn.cursor()

# Загружаем данные из curricula.tsv
load_curricula_from_tsv('curricula.tsv', 'curricula')


cursor.close()
conn.close()

query=INSERT INTO curricula VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
74
row=[0, '2021', '2021', '9', None, None, 'Аника-воин', 'поэзия', 'отечественная', False, False, 6, None, None, 1, None, 'Фольклор', None, 'Не менее 1 баллады народов России и мира по выбору']
19
None


ValueError: 

### Шаг 5. "Проектирование" реляционной базы данных

На данном этапе мы создаем 4 уникальных таблицы (связанных между собой по id).
Я решил разделить атрибуты данных curricula.tsv по трём ключевым критериям: жанр, эпоха и возраст.

Таким образом:

**genres**: содержит информацию о жанрах произведения;

**periods**: содержит информацию об эпохе произведения;

**grades**: содержит информацию о классе/возрасте, в котором изучается произведение;

**works**: мета-таблица с атрибутами id, source_id, title и author;

In [11]:
# Вновь подключаемся к базе данных
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password=getpass('Введите ваш пароль!'),
    database='literature_curricula'
)

cursor = conn.cursor()

# Пишем SQL-скрипт для создания таблиц
create_tables_script = """
-- Создание таблицы genres
CREATE TABLE IF NOT EXISTS genres (
    id INT AUTO_INCREMENT PRIMARY KEY,
    title VARCHAR(255),
    author VARCHAR(255),
    genre VARCHAR(255),
    type VARCHAR(255)
);

-- Создание таблицы periods
CREATE TABLE IF NOT EXISTS periods (
    id INT AUTO_INCREMENT PRIMARY KEY,
    title VARCHAR(255),
    author VARCHAR(255),
    source_year INT,
    century VARCHAR(255),
    source_issue VARCHAR(255)
);

-- Создание таблицы grades
CREATE TABLE IF NOT EXISTS grades (
    id INT AUTO_INCREMENT PRIMARY KEY,
    title VARCHAR(255),
    author VARCHAR(255),
    grade VARCHAR(255),
    notes TEXT
);

-- Создание таблицы works
CREATE TABLE IF NOT EXISTS works (
    id INT AUTO_INCREMENT PRIMARY KEY,
    source_id VARCHAR(255),
    title VARCHAR(255),
    author VARCHAR(255)
);
"""

# Выполненяем!
print("Создание таблиц...")
for statement in create_tables_script.split(';'):
    if statement.strip():
        cursor.execute(statement)
print("Таблицы созданы.")

# Пишем еще один SQL-скрипт, уже для переноса данных
transfer_data_script = """
-- Перенос данных в таблицу genres
INSERT INTO genres (title, author, genre, type)
SELECT title, author, genre, type FROM curricula;

-- Перенос данных в таблицу periods
INSERT INTO periods (title, author, source_year, century, source_issue)
SELECT title, author, source_year, century, source_issue FROM curricula;

-- Перенос данных в таблицу grades
INSERT INTO grades (title, author, grade, notes)
SELECT title, author, grade, notes FROM curricula;

-- Перенос данных в таблицу works
INSERT INTO works (source_id, title, author)
SELECT source_id, title, author FROM curricula;
"""

# Выполняем!
print("Перенос данных...")
for statement in transfer_data_script.split(';'):
    if statement.strip():
        cursor.execute(statement)
print("Данные перенесены.")

conn.commit()
cursor.close()
conn.close()

Введите ваш пароль! ········


Создание таблиц...
Таблицы созданы.
Перенос данных...
Данные перенесены.
Таблица genres содержит 10733 строк.

Пример данных из таблицы genres:
(16384, 'Аника-воин', None, 'поэзия', 'отечественная')
(16385, 'Аника-воин', None, 'поэзия', 'отечественная')
(16386, 'Аника-воин', None, 'поэзия', 'отечественная')
(16387, 'Вольга и Микула Селянинович', None, 'поэзия', 'отечественная')
(16388, 'Гэсэр', None, 'поэзия', 'региональная')
Таблица periods содержит 10733 строк.

Пример данных из таблицы periods:
(16384, 'Аника-воин', None, 2021, None, 'Фольклор')
(16385, 'Аника-воин', None, 2022, None, 'Фольклор')
(16386, 'Аника-воин', None, 2022, None, 'Фольклор')
(16387, 'Вольга и Микула Селянинович', None, 1998, None, 'Из русского фольклора')
(16388, 'Гэсэр', None, 2004, None, 'Литература народов России')
Таблица grades содержит 10733 строк.

Пример данных из таблицы grades:
(16384, 'Аника-воин', None, '6', 'Не менее 1 баллады народов России и мира по выбору')
(16385, 'Аника-воин', None, '6', 'Не 

### Шаг 6. Мини-исследование

В рамках финальной части своей работы я решил определить как изменилось количество иностранных произведений в школьной программе за последние 24 года. С этой целью я...

* Выполнил FULL JOIN таблиц 'periods'(содержит даты включения книги в школьную программу) и 'genres'(содержит параметр 'type'); 
* Разбил представленную выборку данных на три временных периода (1998-2006, 2006-2014, 2007-2016);
* Определил функцию на основе MySQL-скрипта для выбора значения 'зарубежная' в указанных периодах;

In [16]:
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password=getpass('Введите ваш пароль!'),
    database='literature_curricula'
)

cursor = conn.cursor()

# Создаем представление с полным объединением
create_view_query = """
CREATE OR REPLACE VIEW full_join_periods_genres AS
SELECT
    periods.id AS periods_id,
    periods.title AS periods_title,
    periods.author AS periods_author,
    periods.source_year,
    periods.century,
    periods.source_issue,
    genres.id AS genres_id,
    genres.title AS genres_title,
    genres.author AS genres_author,
    genres.genre,
    genres.type
FROM periods
LEFT JOIN genres ON periods.title = genres.title AND periods.author = genres.author
UNION
SELECT
    periods.id AS periods_id,
    periods.title AS periods_title,
    periods.author AS periods_author,
    periods.source_year,
    periods.century,
    periods.source_issue,
    genres.id AS genres_id,
    genres.title AS genres_title,
    genres.author AS genres_author,
    genres.genre,
    genres.type
FROM genres
RIGHT JOIN periods ON genres.title = periods.title AND genres.author = periods.author;
"""

cursor.execute(create_view_query)
conn.commit()
print("Ура, представление создано!")

# Функция для подсчета значений "зарубежная" в колонке type
def count_foreign_type(period_start, period_end):
    query = f"""
    SELECT COUNT(*) AS count_foreign
    FROM full_join_periods_genres
    WHERE type = 'зарубежная' AND source_year BETWEEN {period_start} AND {period_end};
    """
    cursor.execute(query)
    return cursor.fetchone()[0]

# Считаем
count_1998_2006 = count_foreign_type(1998, 2006)
count_2006_2014 = count_foreign_type(2006, 2014)
count_2014_2022 = count_foreign_type(2014, 2022)

print(f"Количество значений 'зарубежная' с 1998-2006 годов: {count_1998_2007}")
print(f"Количество значений 'зарубежная' с 2006-2014 годов: {count_2007_2016}")
print(f"Количество значений 'зарубежная' с 2014-2022 годов: {count_2016_2022}")

cursor.close()
conn.close()

Введите ваш пароль! ········


Ура, представление создано!
Количество значений 'зарубежная' с 1998-2006 годов: 891
Количество значений 'зарубежная' с 2006-2014 годов: 555
Количество значений 'зарубежная' с 2014-2022 годов: 8989


Таким образом, количество произведений иностранной литературы в обязательной школьной программе (почти) вдвое сократилось в период с 2006 по 2014 годы, но существенно выросло (на 8989 единиц!) за последние годы.

### Шаг 7. Дамп и схема БД

Разумеется, это можно было бы сделать и в консоли, но я решил попробовать прописать аналогичные команды здесь для наглядности. Спасибо библиотеке 'subprocess'!

In [20]:
# Подключаемся к БД
username = 'root'
password = getpass('Введите ваш пароль!')
database_name = 'literature_curricula'
dump_file = 'literature_curricula_dump.sql'

# Прописываем команду для дампа
command = f"mysqldump -u {username} -p{password} {database_name} > {dump_file}"

subprocess.run(command, shell=False, check=True)
print(f"Дамп базы данных сохранен в файл {dump_file}.")

# Прописываем команду для схемы
command = f"mysqldump -u {username} -p{password} --no-data {database_name} > {schema_file}"

subprocess.run(command, shell=False, check=True)
print(f"Схема базы данных сохранена в файл {schema_file}.")


Введите ваш пароль! ········


FileNotFoundError: [Errno 2] No such file or directory: 'mysqldump -u root -pMiguel19692000! literature_curricula > literature_curricula_dump.sql'